# Project 3: Quantum Dynamics
##    Eduardo Villaseñor - 
##    Eoin Horgan - 4582322

### Description:
A modelling of the time-dependent Schrödinger equation, taking m = 1, hbar = 1. 

In [1]:
%%latex
\begin{equation}
-i \frac{\partial\psi}{\partial t} = - \frac{\partial^2\psi}{\partial x^2} + V(x) \psi
= H \psi
\end{equation}

<IPython.core.display.Latex object>

The time evolution of the wave function is found using the Crank-Nicolson method:

In [2]:
%%latex
\begin{equation}
\psi^{n+1} = \frac{1-i\Delta t H_D/2}{1+i\Delta t H_D/2} \psi^n
\end{equation}

<IPython.core.display.Latex object>

### Prerequisites: Discrete Matrix Building and Plots

Run the next block before moving onto the demonstrations, below.